Create a folder in Drive. This folder should contain your training weight and classes.txt files that you created with Yolov3.

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [4]:
!git clone https://github.com/mystic123/tensorflow-yolo-v3

Cloning into 'tensorflow-yolo-v3'...
remote: Enumerating objects: 219, done.
remote: Total 219 (delta 0), reused 0 (delta 0), pack-reused 219
Receiving objects: 100% (219/219), 57.63 KiB | 5.24 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [5]:

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
!ln -s /content/gdrive/My \ Drive/ /MyDrive
!ls /MyDrive

ln: target '/MyDrive' is not a directory
ls: cannot access '/MyDrive': No such file or directory


In [7]:
%cd tensorflow-yolo-v3/

/content/tensorflow-yolo-v3


Let's right click for each file you created in Drive, click get link and get the share IDs of the files.

In [13]:
!gdown --id 167OvBj6UB6bL8gF_Hp-G_pPFbkjuwQ-- #classes.txt
!gdown --id 1bItb-EnLf1QnZM434zH7w3ykOpa9n9ec #weights

Downloading...
From: https://drive.google.com/uc?id=167OvBj6UB6bL8gF_Hp-G_pPFbkjuwQ--
To: /content/tensorflow-yolo-v3/classes.txt
100% 273/273 [00:00<00:00, 231kB/s]
Downloading...
From: https://drive.google.com/uc?id=1bItb-EnLf1QnZM434zH7w3ykOpa9n9ec
To: /content/tensorflow-yolo-v3/yolov3_custom_last.weights
100% 247M/247M [00:01<00:00, 195MB/s]


After running the cell below, a .pb file named frozen should be created. If there is an error, there are a few lines of space at the bottom of our classes.txt file.

In [16]:
!python convert_weights_pb.py --class_names "/content/tensorflow-yolo-v3/classes.txt" --weights_file "/content/tensorflow-yolo-v3/yolov3_custom_last.weights" --data_format "NHWC"

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W1214 06:48:20.563391 140058383775616 module_wrapper.py:139] From convert_weights_pb.py:42: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W1214 06:48:20.565326 140058383775616 module_wrapper.py:139] From convert_weights_pb.py:44: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W1214 06:48:20.568203 140058383775616 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.

Let's restart the runtime. and let's upgrade to tensorflow 2.x.

In [1]:
%tensorflow_version 2.x


In [2]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [3]:
import tempfile
import os
import glob

In [4]:
%cd tensorflow-yolo-v3

/content/tensorflow-yolo-v3


In [6]:
import tensorflow.compat.v1 as tf
gf=tf.GraphDef()
m_file=open('/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb','rb')
gf.ParseFromString(m_file.read())

with open('somefile.txt','a') as the_file:
  for n in gf.node:
    the_file.write(n.name+'\n')

file=open('somefile.txt','r')
data=file.readlines()
output=data[len(data)-1]
print("Output array:",output)

file.seek(0)
input=file.readline()
print("Input array", input)

Output array: output_boxes

Input array inputs



In [10]:
converter=tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb',
    input_arrays=['inputs'],
    output_arrays=['output_boxes'],
    input_shapes={'inputs':[1,416,416,3]}
)

converter.optimizations=[tf.lite.Optimize.DEFAULT]

tflite_model=converter.convert()

_,dynamic_tflite_path=tempfile.mkstemp('.tflite')
tflite_model_size=open(dynamic_tflite_path,'wb').write(tflite_model)
tf_model_size=os.path.getsize('/content/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb')
print('Tensorflow model is {} bytes'.format(tf_model_size))
print('TF model is {} bytes'.format(tflite_model_size))
print('Post training dynamic range quantization saves {} bytes'.format(tf_model_size-tflite_model_size))

Tensorflow model is 246986268 bytes
TF model is 62474752 bytes
Post training dynamic range quantization saves 184511516 bytes


In [11]:
!ls -lh {dynamic_tflite_path}

-rw------- 1 root root 60M Dec 14 07:14 /tmp/tmpkngoal_k.tflite


In [12]:
interpreter=tf.lite.Interpreter(model_path='/content/gdrive/MyDrive/TFLite_Yolov3/detect.tflite')

input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()

In our tmp folder, our weight with .tflite extension has been created.

In [13]:
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'inputs',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 416, 416,   3], dtype=int32),
  'shape_signature': array([  1, 416, 416,   3], dtype=int32),
  'sparsity_parameters': {}}]

In [14]:
output_details

[{'dtype': numpy.float32,
  'index': 407,
  'name': 'output_boxes',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([    1, 10647,    29], dtype=int32),
  'shape_signature': array([    1, 10647,    29], dtype=int32),
  'sparsity_parameters': {}}]